In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install tfa-nightly
!pip install import-ipynb
import import_ipynb
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.layers import *
import os
import scipy.io as sio
import matplotlib.pyplot as plt
import tensorflow_addons as tfa
from tqdm import tqdm
from fastprogress import master_bar, progress_bar
%cd '/content/drive/MyDrive/Toan/LMSLoss'

/content/drive/.shortcut-targets-by-id/12FMN0xPZzeIH25Go6ZQUrtzx9sf8Trq9/LMSLoss


In [ ]:
%run UnetModel.ipynb
# %run MumfordLoss.ipynb
%run augment_data.ipynb




Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 1) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 128, 128, 64) 576         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 128, 128, 64) 256         conv2d[0][0]                     
__________________________________________________________________________________________________
swish (Swish)                   (None, 128, 128, 64) 0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [ ]:
import keras.backend as K

def dice_coef(y_true, y_pred, smooth=1e-10):
    '''Average dice coefficient per batch.'''
    axes = (1,2,3)
    intersection = K.sum(y_true * y_pred, axis=axes)
    summation = K.sum(y_true + y_pred, axis=axes)
    return K.mean((2.0 * intersection + smooth) / (summation + smooth), axis=0)

def jaccard_coef(y_true, y_pred, smooth=1e-10):
    '''Average jaccard coefficient per batch.'''
    axes = (1,2,3)
    intersection = K.sum(y_true * y_pred, axis=axes)
    union = K.sum(y_true + y_pred, axis=axes) - intersection
    return K.mean( (intersection + smooth) / (union + smooth), axis=0)
def dice_coef_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred, 0.0)

In [ ]:
import keras.backend as K

def dice_coef(y_true, y_pred, smooth=1e-10):
    '''Average dice coefficient per batch.'''
    axes = (1,2,3)
    intersection = K.sum(y_true * y_pred, axis=axes)
    summation = K.sum(y_true + y_pred, axis=axes)
    return K.mean((2.0 * intersection + smooth) / (summation + smooth), axis=0)

def dice_coef_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred, 0.0)

def g_loss(y_true, y_pred,lambda):
    return bce_loss + lambda * dice_coef_loss(y_true, y_pred)

#bce loss =  lograrit cua dau ra cua d_model khi dau vao la (inputs, y_pred)

gan_model.compile(loss = g_loss ,optimizer=tf.keras.optimizers.Nadam(0.001)

In [ ]:
x = np.asarray(np.load('./dataACDCA/x_crop_128.npy'),np.float32)
y1 = np.load('./dataACDCA/y_crop_128.npy')
label_endo = np.where(y1 == 2, 1.0, 0.0)
label_epi =  np.where(y1 == 1, 1.0, 0.0) + label_endo

In [ ]:
#@title type of train { form-width: "500px" }
type_of_training = "endo" #@param ["endo", "epi"]
if type_of_training == "endo":
  y = label_endo
else:
  y = label_epi
np.random.seed(123)
len_dev = x.shape[0]//5
len_train = x.shape[0] - len_dev
rand_index = np.random.permutation(x.shape[0])
x_test = x[rand_index[:len_dev]].astype(np.float32)
y_test = y[rand_index[:len_dev]].astype(np.float32)
x_train = x[rand_index[len_dev:]].astype(np.float32)
y_train = y[rand_index[len_dev:]].astype(np.float32)


In [ ]:
BATCH_SIZE = 32
buffer_size = x_train.shape[0]

@tf.function
def gen_image(image,mask):
  # image, mask = brightness(image,mask)
  #image, mask = saturation(image,mask)
  image, mask = flip(image,mask)
  image, mask = rotate_image(image,mask)
  image, mask = normalize(image, mask)
  return image, mask
train_dataset = tf.data.Dataset.from_tensor_slices((x_train,y_train)).shuffle(buffer_size).map(gen_image).batch(BATCH_SIZE)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test,y_test)).map(normalize).batch(BATCH_SIZE*2)

In [ ]:
S = seg_net((128,128,1),out_channels=1)
S.compile(loss = 'binary_crossentropy',optimizer=tf.keras.optimizers.Nadam(0.001),metrics=[dice_coef, jaccard_coef])

In [ ]:
mcp = ModelCheckpoint("./weightACDCA_BCE/weight"+type_of_training+"{val_dice_coef:.4f}.h5",mode='max',monitor='val_dice_coef',verbose=1,save_best_only=True,save_weights_only=True)
earlystop= EarlyStopping(monitor='val_dice_coef',patience=22,mode='max',verbose=1)
rlr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_dice_coef', factor=0.5, mode='max', patience=10, min_lr=1e-6, verbose=1)
history = S.fit(train_dataset,validation_data=test_dataset,
	    epochs=300,callbacks=[earlystop,mcp,rlr],verbose=1)

Epoch 1/300
48/48 [==============================] - ETA: 0s - loss: 0.2378 - dice_coef: 0.1283 - jaccard_coef: 0.0727
Epoch 00001: val_dice_coef improved from -inf to 0.09519, saving model to ./weightACDCA_BCE/weightendo0.0952.h5
48/48 [==============================] - 48s 991ms/step - loss: 0.2378 - dice_coef: 0.1283 - jaccard_coef: 0.0727 - val_loss: 2.3021 - val_dice_coef: 0.0952 - val_jaccard_coef: 0.0515
Epoch 2/300
48/48 [==============================] - ETA: 0s - loss: 0.0873 - dice_coef: 0.3352 - jaccard_coef: 0.2219
Epoch 00002: val_dice_coef improved from 0.09519 to 0.21120, saving model to ./weightACDCA_BCE/weightendo0.2112.h5
48/48 [==============================] - 40s 836ms/step - loss: 0.0873 - dice_coef: 0.3352 - jaccard_coef: 0.2219 - val_loss: 0.4733 - val_dice_coef: 0.2112 - val_jaccard_coef: 0.1308
Epoch 3/300
48/48 [==============================] - ETA: 0s - loss: 0.0701 - dice_coef: 0.4282 - jaccard_coef: 0.3027
Epoch 00003: val_dice_coef improved from 0.21120

In [ ]:
sio.savemat('./weightACDCA_BCE/historyACDCA'+type_of_training+'_DL.mat',history.history)